In [1]:
!pip install transformers diffusers accelerate
import transformers, diffusers
transformers.CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14").save_pretrained("./tokenizers/")
transformers.CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").save_pretrained("./models/text/")
diffusers.UNet2DConditionModel.from_pretrained("prompthero/openjourney", subfolder = "unet").save_pretrained("./models/unet/")
diffusers.AutoencoderKL.from_pretrained("prompthero/openjourney", subfolder = "vae").save_pretrained("./models/vae/")

c:\Users\Austin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch, tqdm, PIL, numpy, datetime, diffusers, transformers
def diffuse(prompt, takes = 1):
    latents = (scheduler.sigmas[0] * torch.randn(takes, 4, 64, 64)).to("cuda")
    index = 0
    with torch.no_grad():
        embeddings = torch.cat([text(tokenizer(takes * [""], padding = "max_length", truncation = True, return_tensors = "pt").input_ids.to("cuda"))["last_hidden_state"], text(tokenizer(takes * [prompt], padding = "max_length", truncation = True, return_tensors = "pt").input_ids.to("cuda"))["last_hidden_state"]])
        with torch.autocast("cuda"):
            for value in tqdm.tqdm(scheduler.timesteps):
                noise = unet(torch.cat([latents, latents]) / (scheduler.sigmas[index] ** 2 + 1) ** .5, value, embeddings)["sample"].chunk(2)
                latents = scheduler.step(11 * noise[1] - 10 * noise[0], value, latents)["prev_sample"]
                index += 1
        index = 0
        truncation = prompt[: 100]
        images = PIL.Image.new("RGB", [512 * takes, 512])
        for value in (127.5 * vae.decode(latents / .18215)["sample"].clamp(-1, 1).detach().cpu().permute(0, 2, 3, 1).numpy() + 127.5).astype(numpy.uint8):
            images.paste(PIL.Image.fromarray(value), [512 * index, 0])
            index += 1
    return images
scheduler = diffusers.LMSDiscreteScheduler(beta_start = .00085, beta_end = .012, beta_schedule = "scaled_linear")
scheduler.set_timesteps(28)
tokenizer = transformers.CLIPTokenizer.from_pretrained("./tokenizers/")
text = transformers.CLIPTextModel.from_pretrained("./models/text/").to("cuda")
unet = diffusers.UNet2DConditionModel.from_pretrained("./models/unet/").to("cuda")
vae = diffusers.AutoencoderKL.from_pretrained("./models/vae/").to("cuda")

In [ ]:
diffuse("masterpiece, best quality, abstract painting of a lush pond landscape, photorealistic, rtx on, 8k, by greg ratakowski", 4)